# Combine data sources into single dataframe

In [1]:
import numpy as np
import pandas as pd
from pymap3d.vincenty import vdist

### Read airports data

In [2]:
# Column names in airports.dat
columns = [
    "ID",
    "Name",
    "City",
    "Country",
    "IATA",
    "ICAO",
    "Latitude",  # Degrees
    "Longitude",  # Degrees
    "Altitude",  # Feet
    "UTC Offset",
    "DST",
    "Timezone",
    "Type",
    "Source",
]

# Columns to keep
useCols = [
    "Name",
    "City",
    "Country",
    "IATA",
    "Latitude",
    "Longitude",
    "Timezone",
    "UTC Offset",
]

df1 = pd.read_csv("data/airports.dat", names=columns, usecols=useCols, na_values=["\\N"])
df1.dropna(axis=0, inplace=True)
df1.set_index("IATA", inplace=True)
df1.head()

,Name,City,Country,Latitude,Longitude,UTC Offset,Timezone
IATA,,,,,,,
GKA,Goroka Airport,Goroka,Papua New Guinea,-6.081690,145.391998,10.0,Pacific/Port_Moresby
MAG,Madang Airport,Madang,Papua New Guinea,-5.207080,145.789001,10.0,Pacific/Port_Moresby
HGU,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,-5.826790,144.296005,10.0,Pacific/Port_Moresby
LAE,Nadzab Airport,Nadzab,Papua New Guinea,-6.569803,146.725977,10.0,Pacific/Port_Moresby
POM,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,-9.443380,147.220001,10.0,Pacific/Port_Moresby


### Read Enplanements data

In [3]:
df2 = pd.read_excel("data/cy20-commercial-service-enplanements.xlsx")
df2.dropna(subset=["Locid"], inplace=True)
df2.rename(columns={"Locid": "IATA"}, inplace=True)
df2.set_index("IATA", inplace=True)
df2.drop(["City", "Airport Name"], axis=1, inplace=True)
df2.head()

,Rank,RO,ST,S/L,Hub,CY 20 Enplanements,CY 19 Enplanements,% Change
IATA,,,,,,,,
ATL,1,SO,GA,P,L,20559866,53505795.0,-0.6157
DFW,2,SW,TX,P,L,18593421,35778573.0,-0.4803
DEN,3,NM,CO,P,L,16243216,33592945.0,-0.5165
ORD,4,GL,IL,P,L,14606034,40871223.0,-0.6426
LAX,5,WP,CA,P,L,14055777,42939104.0,-0.6727


### Merge dataframes

In [4]:
df = df2.merge(df1, how="inner", on="IATA")
df.head()

,Rank,RO,ST,S/L,Hub,CY 20 Enplanements,CY 19 Enplanements,% Change,Name,City,Country,Latitude,Longitude,UTC Offset,Timezone
IATA,,,,,,,,,,,,,,,
ATL,1,SO,GA,P,L,20559866,53505795.0,-0.6157,Hartsfield Jackson Atlanta International Airport,Atlanta,United States,33.636700,-84.428101,-5.0,America/New_York
DFW,2,SW,TX,P,L,18593421,35778573.0,-0.4803,Dallas Fort Worth International Airport,Dallas-Fort Worth,United States,32.896801,-97.038002,-6.0,America/Chicago
DEN,3,NM,CO,P,L,16243216,33592945.0,-0.5165,Denver International Airport,Denver,United States,39.861698,-104.672997,-7.0,America/Denver
ORD,4,GL,IL,P,L,14606034,40871223.0,-0.6426,Chicago O'Hare International Airport,Chicago,United States,41.978600,-87.904800,-6.0,America/Chicago
LAX,5,WP,CA,P,L,14055777,42939104.0,-0.6727,Los Angeles International Airport,Los Angeles,United States,33.942501,-118.407997,-8.0,America/Los_Angeles


In [5]:
IATA = sorted(df.index)
numAirports = len(IATA)

In [6]:
# Convert meter to mile
def meter_to_mile(meters):
    return meters * (100 / 2.54) / (12 * 5280)


# Compute distance from one airport to many
def batch_distance(oneAirport, manyAirports):
    oneAirport = oneAirport.upper()
    if type(manyAirports) is str:
        manyAirports = [manyAirports]
    manyAirports = list(map(lambda x: x.upper(), manyAirports))
    src = df.loc[oneAirport]
    dst = df.loc[manyAirports]
    dist_m, azimuth_deg = vdist(
        src.Latitude, src.Longitude, dst.Latitude, dst.Longitude
    )
    return meter_to_mile(dist_m)


# Compute distance from one airport to another
def airport_distance(dst, src):
    if src == dst:
        return 0
    if src not in df.index:
        raise ValueError('IATA code "%s" is not in dataframe' % src)
    if dst not in df.index:
        raise ValueError('IATA code "%s" is not in dataframe' % dst)
    src = df.loc[src]
    dst = df.loc[dst]
    dist_m, azimuth_deg = vdist(
        src.Latitude, src.Longitude, dst.Latitude, dst.Longitude
    )
    dist_miles = meter_to_mile(dist_m)
    return dist_miles

### Compute pairwise distances

In [7]:
adjMatrix = np.zeros((numAirports, numAirports))
for i, src in enumerate(IATA):
    adjMatrix[i, :i] = batch_distance(src, IATA[:i])
adjMatrix += adjMatrix.T

In [8]:
adjdf = pd.DataFrame(adjMatrix, index=IATA, columns=IATA)
adjdf.head()

,ABE,ABI,ABQ,ABR,ABY,ACK,ACT,ACV,ACY,ADQ,...,VEL,VLD,VPS,VQS,WRG,WST,WYS,XNA,YAK,YKM
ABE,0.000000,1457.556140,1738.257161,1204.924470,795.974016,284.485245,1363.662660,2517.487534,94.277461,3469.707215,...,1781.943347,809.803686,937.199464,1662.900971,2735.307498,196.137905,1823.620056,1064.143711,2993.372708,2267.209906
ABI,1457.556140,0.000000,438.000801,901.895168,910.811338,1741.099801,154.106833,1472.967462,1483.248401,3009.179034,...,777.748934,972.938046,787.822106,2345.406852,2278.319262,1653.693194,1046.938571,406.820753,2614.777107,1473.957072
ABQ,1738.257161,438.000801,0.000000,837.496474,1316.705988,2020.693728,591.477869,1036.664592,1782.301932,2619.097274,...,404.874934,1384.188210,1208.415706,2777.738367,1911.677862,1930.090792,707.197699,696.959155,2251.048544,1076.879347
ABR,1204.924470,901.895168,837.496474,0.000000,1227.268336,1449.890099,956.639618,1327.024719,1281.362315,2402.930316,...,659.184233,1300.852045,1216.548418,2670.758064,1643.499217,1362.816840,622.844720,668.049536,1949.606640,1063.947303
ABY,795.974016,910.811338,1316.705988,1227.268336,0.000000,1031.859259,768.496555,2300.078332,768.679478,3625.469268,...,1538.588847,75.151096,156.151474,1493.641074,2865.875354,964.741311,1712.664928,666.324976,3176.502126,2186.437532


### Merge into dataframe

In [9]:
df = adjdf.merge(df, right_on="IATA", left_index=True)
df.head()

,ABE,ABI,ABQ,ABR,ABY,ACK,ACT,ACV,ACY,ADQ,...,CY 20 Enplanements,CY 19 Enplanements,% Change,Name,City,Country,Latitude,Longitude,UTC Offset,Timezone
IATA,,,,,,,,,,,,,,,,,,,,,
ABE,0.000000,1457.556140,1738.257161,1204.924470,795.974016,284.485245,1363.662660,2517.487534,94.277461,3469.707215,...,187530,434007.0,-0.5679,Lehigh Valley International Airport,Allentown,United States,40.652100,-75.440804,-5.0,America/New_York
ABI,1457.556140,0.000000,438.000801,901.895168,910.811338,1741.099801,154.106833,1472.967462,1483.248401,3009.179034,...,44202,81813.0,-0.4597,Abilene Regional Airport,Abilene,United States,32.411301,-99.681900,-6.0,America/Chicago
ABQ,1738.257161,438.000801,0.000000,837.496474,1316.705988,2020.693728,591.477869,1036.664592,1782.301932,2619.097274,...,868922,2641450.0,-0.671,Albuquerque International Sunport,Albuquerque,United States,35.040199,-106.609001,-7.0,America/Denver
ABR,1204.924470,901.895168,837.496474,0.000000,1227.268336,1449.890099,956.639618,1327.024719,1281.362315,2402.930316,...,14854,29564.0,-0.4976,Aberdeen Regional Airport,Aberdeen,United States,45.449100,-98.421799,-6.0,America/Chicago
ABY,795.974016,910.811338,1316.705988,1227.268336,0.000000,1031.859259,768.496555,2300.078332,768.679478,3625.469268,...,17222,41268.0,-0.5827,Southwest Georgia Regional Airport,Albany,United States,31.535500,-84.194504,-5.0,America/New_York


### Save

In [10]:
df.to_pickle("data/airportdf.pkl")